In [1]:
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

from utils import load_test_data

In [2]:
ensembling_models = {
    'DenseNet201-Fine-Tune': {'img_size': 224, 'mean_proc': 'DenseNet'},
    'Xception-Fine-Tune': {'img_size': 299,  'mean_proc': None}
}

In [3]:
for key in ensembling_models.keys():
    print(key)
    img_size = ensembling_models[key]['img_size']
    mean_proc = ensembling_models[key]['mean_proc']
    model = load_model('./saved_models/{}.h5'.format(key))
    X_test, X_id = load_test_data(Gray2RGB=True, mean_proc=mean_proc, img_size=img_size)
    ensembling_models[key]['y_test_pred_prob'] = model.predict(X_test)

DenseNet201-Fine-Tune
Xception-Fine-Tune


In [8]:
num_models = float(len(ensembling_models.keys()))
y_test_pred_prob = 0
for key in ensembling_models.keys():
    y_test_pred_prob += ensembling_models[key]['y_test_pred_prob']

y_test_pred_prob /= num_models
print(y_test_pred_prob)

[[4.19536280e-03 8.15565072e-05 1.70062500e-04 ... 6.46582572e-03
  7.56306836e-05 1.49153129e-04]
 [1.01110396e-04 1.69427338e-04 1.65067206e-04 ... 1.81520736e-04
  3.47291934e-04 8.78640276e-05]
 [4.70937433e-04 3.91363050e-04 1.03707515e-01 ... 4.50395368e-04
  4.95616831e-02 6.16740028e-04]
 ...
 [1.76292560e-05 3.00605516e-05 3.23097993e-05 ... 2.08512174e-05
  1.23088448e-05 9.17932812e-06]
 [1.42941950e-04 1.55188434e-04 4.76466084e-04 ... 1.14239563e-04
  2.40394234e-04 3.12511344e-04]
 [1.95298315e-04 1.15272813e-02 7.60793965e-03 ... 7.64945362e-05
  2.09645601e-03 7.88951933e-04]]


In [5]:
y_test_pred = y_test_pred_prob.argmax(axis=-1)

In [6]:
import numpy as np
import pandas as pd
X_test, X_id = load_test_data(Gray2RGB=True, mean_proc='VGG16_ImageNet', img_size=img_size)
y_test_pred_df = pd.DataFrame({'id': np.array(X_id), 'class':y_test_pred}).sort_values(by='id')

In [7]:
y_test_pred_df.to_csv('./submissions/ensemble.csv', index=False)